# Introduction

This final project is designated to help people moving into Paris, France discover which Borough is more Eco-friendly, so they can weigh that consideration in their decision where to rent an apartment.
The main goal is to Determine which neighborhood has "Green" criteria, based on Foresquare data and air quality data within each borough.

# Data

In order to answer the question above, data on the twenty arrondissements of Paris include latitude, longitude, Gardens, Parks, Vegetarian/Vegan restaurants, Organic groceries and Air Quality index in each borough are necessary. 

1. Paris data containing the boroughs, latitudes, and longitudes will be obtained from the data source: https://opendata.paris.fr/explore/dataset/arrondissements/table/?location=12,48.85889,2.34692&basemap=jawg.streets

2. Paris data containing historical air quality index for 2019 will be taken from the Breezometer API : https://breezometer.com/historical-air-quality-data using the Request library in Python.

3. All data related to locations of Public open spaces and Eco-friendly businesses will be obtained via the FourSquare API by the Request library in Python.